# SEIRX model runs for no intervention measures

In [18]:
import networkx as nx
import pandas as pd
from os.path import join
import json

# parallelisation functionality
from multiprocess import Pool
import psutil
from tqdm import tqdm

# custom functionality
from scseirx.model_nursing_home import SEIRX_nursing_home # agent-based simulation
from scseirx import analysis_functions as af # custom analysis functions
import data_creation_functions as dcf

from importlib import reload

## Parameters

### Simulation parameters and measures in place

In [19]:
with open('params/no_intervention_simulation_parameters.json') as json_file:
    simulation_params = json.load(json_file)
    
with open('params/no_intervention_measures.json') as json_file:
    measures = json.load(json_file)


In [20]:
# paths for I/O
contact_network_src = '../data/contact_networks'
ensmbl_dst = '../data/simulation_results/ensembles'

### Screening parameters

In [21]:
# specifies, whether the index case will be introduced via an employee or a 
# resident
index_cases = ['employee', 'resident']

# ensemble_size
N_runs = 1000

screening_params = [(None, j, 0, 0, N_runs) for j in index_cases ]

print('there are {} parameter combinations'.format(len(screening_params)))

there are 2 parameter combinations


In [22]:
def run(param_list):
    '''
    Runs an ensemble of simulations and collects observable statistics. To be 
    run in parallel on many workers. Note: I/O paths and the number of runs per 
    ensemble hare hard coded here, because I only want to pass the parameter 
    values that are being screened in the simulation run to the function via the
    parallel processing interface.
    
    Parameters:
    -----------
    param_list : iterable
        Iterable that contains the values for the parameters test_type, 
        index_case, e_screen_range and r_screen_range that are passed to the
        simulation.
    N_runs : integer
        Number of runs per ensemble.
        
    Returns:
    --------
    row : dictionary
        Dictionary of the ensemble statistics of the observables.
    '''    

    # extract the simulation parameters from the parameter list
    test_type, index_case, e_screen_range, r_screen_range, N_runs = param_list

    # run the ensemble with the given simulation parameters 
    ensemble_results = dcf.run_ensemble(N_runs, test_type, index_case, 
                e_screen_range, r_screen_range, measures,
                simulation_params, contact_network_src, ensmbl_dst)

    # calculate the ensemble statistics
    row = dcf.evaluate_ensemble(ensemble_results, test_type, index_case, 
                                e_screen_range, r_screen_range)

    return row

In [23]:
%%time
number_of_cores = psutil.cpu_count(logical=True) - 2
pool = Pool(number_of_cores)

rows = []
for row in tqdm(pool.imap_unordered(func=run, iterable=screening_params),
                total=len(screening_params)):
        rows.append(row)

# turn off your parallel workers 
pool.close()

# format and dump the results to disk
dst = '../data/simulation_results'
results = pd.DataFrame()
for row in rows:
    results = results.append(row, ignore_index=True)
    
results.reset_index()
index_cols = ['index_case']
other_cols = [c for c in results.columns if c not in index_cols]
results = results[index_cols + other_cols]

results.to_csv(join(dst,'simulations_no_measures_{}.csv'\
                   .format(N_runs)), index=False)
results.head(3)

100%|██████████| 2/2 [08:29<00:00, 254.90s/it]

CPU times: user 146 ms, sys: 157 ms, total: 303 ms
Wall time: 8min 30s


,index_case,N_diagnostic_tests_0.025,N_diagnostic_tests_0.10,N_diagnostic_tests_0.25,N_diagnostic_tests_0.75,N_diagnostic_tests_0.90,N_diagnostic_tests_0.975,N_diagnostic_tests_mean,N_diagnostic_tests_median,N_diagnostic_tests_std,...,transmissions_std,undetected_infections_0.025,undetected_infections_0.10,undetected_infections_0.25,undetected_infections_0.75,undetected_infections_0.90,undetected_infections_0.975,undetected_infections_mean,undetected_infections_median,undetected_infections_std
0,resident,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,17.477081,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,employee,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,17.512664,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
